In [14]:
import yfinance as yf
import pandas as pd
import numpy as np
from functools import reduce
import os

OBS: BTC has only data from 2014-09-17

In [ ]:
# Get Gold, BTC and SPY data. Gold spot and SPY are proxy for Gold and S&P 500 index, respectively.
gold = yf.Ticker("GLD")
data_gold = gold.history(period="11y")
df_gold = pd.DataFrame(data_gold)
print(df_gold)

btc = yf.Ticker("BTC-USD")
data_btc = btc.history(period="15y")
df_btc = pd.DataFrame(data_btc)
print(df_btc)


sp500 = yf.Ticker("SPY")
data_sp500 = sp500.history(period="11y")
df_sp500 = pd.DataFrame(data_sp500)
print(df_sp500)

                                 Open        High         Low       Close  \
Date                                                                        
2014-10-06 00:00:00-04:00  115.160004  116.309998  114.940002  116.029999   
2014-10-07 00:00:00-04:00  116.309998  116.629997  116.089996  116.360001   
2014-10-08 00:00:00-04:00  117.029999  117.709999  115.849998  117.470001   
2014-10-09 00:00:00-04:00  117.860001  118.260002  117.489998  117.639999   
2014-10-10 00:00:00-04:00  117.699997  117.739998  117.190002  117.589996   
...                               ...         ...         ...         ...   
2025-09-29 00:00:00-04:00  351.790009  352.829987  351.279999  352.459991   
2025-09-30 00:00:00-04:00  351.130005  355.570007  350.869995  355.470001   
2025-10-01 00:00:00-04:00  356.170013  356.970001  354.790009  356.029999   
2025-10-02 00:00:00-04:00  358.260010  358.670013  351.399994  354.790009   
2025-10-03 00:00:00-04:00  357.119995  358.140015  355.799988  357.640015   

In [16]:
# Function to fix date format as pure date objects
def fix_date(df):
    df = df.copy()
    df = df.reset_index()  
    df['Date'] = pd.to_datetime(df['Date']).dt.date  
    return df

# Apply to DataFrames
df_gold = fix_date(df_gold)
df_btc = fix_date(df_btc)
df_sp500 = fix_date(df_sp500)

print(df_gold.info())

print(df_gold)
print(df_btc)
print(df_sp500)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2766 entries, 0 to 2765
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           2766 non-null   object 
 1   Open           2766 non-null   float64
 2   High           2766 non-null   float64
 3   Low            2766 non-null   float64
 4   Close          2766 non-null   float64
 5   Volume         2766 non-null   int64  
 6   Dividends      2766 non-null   float64
 7   Stock Splits   2766 non-null   float64
 8   Capital Gains  2766 non-null   float64
dtypes: float64(7), int64(1), object(1)
memory usage: 194.6+ KB
None
            Date        Open        High         Low       Close    Volume  \
0     2014-10-06  115.160004  116.309998  114.940002  116.029999   7644100   
1     2014-10-07  116.309998  116.629997  116.089996  116.360001   5274700   
2     2014-10-08  117.029999  117.709999  115.849998  117.470001  10670900   
3     2014-10-09  117.860001 

In [17]:
# Rename columns to avoid conflicts during merge
gold = df_gold.rename(columns=lambda x: x + "_Gold" if x != "Date" else x)
btc = df_btc.rename(columns=lambda x: x + "_BTC" if x != "Date" else x)
sp500 = df_sp500.rename(columns=lambda x: x + "_SP500" if x != "Date" else x)

# Merge 
df_merged = gold.merge(btc, on="Date", how="inner") \
                .merge(sp500, on="Date", how="inner")

df_merged.info()
print(df_merged)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2766 entries, 0 to 2765
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Date                 2766 non-null   object 
 1   Open_Gold            2766 non-null   float64
 2   High_Gold            2766 non-null   float64
 3   Low_Gold             2766 non-null   float64
 4   Close_Gold           2766 non-null   float64
 5   Volume_Gold          2766 non-null   int64  
 6   Dividends_Gold       2766 non-null   float64
 7   Stock Splits_Gold    2766 non-null   float64
 8   Capital Gains_Gold   2766 non-null   float64
 9   Open_BTC             2766 non-null   float64
 10  High_BTC             2766 non-null   float64
 11  Low_BTC              2766 non-null   float64
 12  Close_BTC            2766 non-null   float64
 13  Volume_BTC           2766 non-null   int64  
 14  Dividends_BTC        2766 non-null   float64
 15  Stock Splits_BTC     2766 non-null   f

OBS: After merging all dataframes it's a must to convert them to monthly values to match monthly economic indicatores.

Aggregation rules preserve financial meaning:

Open = first of the month: reflects starting valuation.

High = maximum: captures peak risk/reward.

Low = minimum: captures worst-case risk.

Close = last of the month: used for return calculations.

Volume = sum: shows total trading activity.

Dividends / Capital Gains = sum: reflects total cash distributions for the month.

In [18]:
base_path = os.getcwd()
file_path = os.path.join("..", "data", "US_ECO.csv")

#Add economic indicators data
eco = pd.read_csv(file_path)

print(eco)
print(eco.info())

# Convert the 'Date' column in eco to datetime
eco['Date'] = pd.to_datetime(eco['Date'], format="%Y:%m")
print(eco['Date'])

# Ensure 'Date' is datetime
eco['Date'] = pd.to_datetime(eco['Date'])

# Drop rows before 2014-09-01
eco = eco[eco['Date'] >= '2014-09-01'].copy()

# Convert all columns except 'Date' to float 
cols_to_convert = eco.columns.difference(['Date'])
eco[cols_to_convert] = eco[cols_to_convert].apply(lambda x: x.str.replace(',', '.').astype(float))


print(eco.info())
print(eco.head())


        Date    P_I   EU_H    C_H   SO_I  CFNAI CFNAI_MA3 DIFFUSION
0    1967:03  -0,26  -0,06  -0,01  -0,01  -0,35       NaN       NaN
1    1967:04   0,17  -0,10   0,01  -0,08   0,00       NaN       NaN
2    1967:05  -0,43   0,01  -0,06  -0,08  -0,56     -0,30     -0,17
3    1967:06  -0,09   0,03   0,08   0,01   0,03     -0,18     -0,15
4    1967:07  -0,19   0,10  -0,07  -0,16  -0,33     -0,28     -0,23
..       ...    ...    ...    ...    ...    ...       ...       ...
696  2025:03  -0,02  -0,07   0,09   0,16   0,16      0,04     -0,01
697  2025:04  -0,18  -0,02  -0,03  -0,18  -0,41      0,04     -0,08
698  2025:05  -0,06  -0,05  -0,10   0,02  -0,18     -0,14     -0,31
699  2025:06   0,01  -0,08  -0,01  -0,10  -0,18     -0,26     -0,40
700  2025:07  -0,10  -0,06   0,00  -0,02  -0,19     -0,18     -0,31

[701 rows x 8 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 701 entries, 0 to 700
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     -

In [19]:

df_merged['Date'] = pd.to_datetime(df_merged['Date'])
df_merged = df_merged.set_index('Date')


agg_dict = {
    'Open_Gold': 'first', 'High_Gold': 'max', 'Low_Gold': 'min', 'Close_Gold': 'last', 'Volume_Gold': 'sum', 
    'Dividends_Gold': 'sum', 'Stock Splits_Gold': 'sum',
    'Open_BTC': 'first', 'High_BTC': 'max', 'Low_BTC': 'min', 'Close_BTC': 'last', 'Volume_BTC': 'sum',
    'Dividends_BTC': 'sum', 'Stock Splits_BTC': 'sum',
    'Open_SP500': 'first', 'High_SP500': 'max', 'Low_SP500': 'min', 'Close_SP500': 'last', 'Volume_SP500': 'sum',
    'Dividends_SP500': 'sum', 'Stock Splits_SP500': 'sum', 'Capital Gains_SP500': 'sum'
}



df_monthly = df_merged.resample('ME').agg(agg_dict).reset_index()

df_monthly['YearMonth'] = df_monthly['Date'].dt.to_period('M')

for col in ['Close_Gold', 'Close_BTC', 'Close_SP500']:
    df_monthly[col + '_Return'] = df_monthly[col].pct_change()


eco_monthly = eco.set_index('Date').resample('ME').last().reset_index()
eco_monthly['YearMonth'] = eco_monthly['Date'].dt.to_period('M')

df_analysis = pd.merge(df_monthly, eco_monthly, on='YearMonth', how='inner', suffixes=('', '_econ'))


print(df_analysis.head())
print(df_analysis.info())


        Date   Open_Gold   High_Gold    Low_Gold  Close_Gold  Volume_Gold  \
0 2014-10-31  115.160004  120.500000  111.660004  112.660004    130513900   
1 2014-11-30  112.389999  115.959999  109.669998  112.110001    147594200   
2 2014-12-31  113.800003  118.989998  112.410004  113.580002    153722200   
3 2015-01-31  112.489998  125.580002  112.320000  123.449997    198034100   
4 2015-02-28  121.839996  123.160004  114.290001  116.160004    125686200   

   Dividends_Gold  Stock Splits_Gold    Open_BTC    High_BTC  ...  \
0             0.0                0.0  320.389008  411.697998  ...   
1             0.0                0.0  325.569000  457.092987  ...   
2             0.0                0.0  378.248993  384.037994  ...   
3             0.0                0.0  314.079010  315.838989  ...   
4             0.0                0.0  226.490997  256.653015  ...   

   Close_BTC_Return  Close_SP500_Return  Date_econ   P_I  EU_H   C_H  SO_I  \
0               NaN                 NaN 2014

In [20]:
# Drop unwanted columns
df_analysis = df_analysis.drop(
    columns=[col for col in df_analysis.columns 
             if "Dividends" in col or "Splits" in col or 
                col == "Capital Gains_SP500" or col.endswith("_econ")]
)

print(df_analysis.head())
print(df_analysis.info())

df_analysis

        Date   Open_Gold   High_Gold    Low_Gold  Close_Gold  Volume_Gold  \
0 2014-10-31  115.160004  120.500000  111.660004  112.660004    130513900   
1 2014-11-30  112.389999  115.959999  109.669998  112.110001    147594200   
2 2014-12-31  113.800003  118.989998  112.410004  113.580002    153722200   
3 2015-01-31  112.489998  125.580002  112.320000  123.449997    198034100   
4 2015-02-28  121.839996  123.160004  114.290001  116.160004    125686200   

     Open_BTC    High_BTC     Low_BTC   Close_BTC  ...  Close_Gold_Return  \
0  320.389008  411.697998  302.559998  338.321014  ...                NaN   
1  325.569000  457.092987  325.076996  376.446991  ...          -0.004882   
2  378.248993  384.037994  304.231995  320.192993  ...           0.013112   
3  314.079010  315.838989  171.509995  226.425003  ...           0.086899   
4  226.490997  256.653015  214.725006  253.828003  ...          -0.059052   

   Close_BTC_Return  Close_SP500_Return   P_I  EU_H   C_H  SO_I  CFNAI  \


,Date,Open_Gold,High_Gold,Low_Gold,Close_Gold,Volume_Gold,Open_BTC,High_BTC,Low_BTC,Close_BTC,...,Close_Gold_Return,Close_BTC_Return,Close_SP500_Return,P_I,EU_H,C_H,SO_I,CFNAI,CFNAI_MA3,DIFFUSION
0,2014-10-31,115.160004,120.500000,111.660004,112.660004,130513900,320.389008,411.697998,302.559998,338.321014,...,NaN,NaN,NaN,-0.03,0.11,-0.03,-0.02,0.04,-0.06,0.01
1,2014-11-30,112.389999,115.959999,109.669998,112.110001,147594200,325.569000,457.092987,325.076996,376.446991,...,-0.004882,0.112692,0.027472,0.29,0.05,-0.05,-0.02,0.27,0.11,0.12
2,2014-12-31,113.800003,118.989998,112.410004,113.580002,153722200,378.248993,384.037994,304.231995,320.192993,...,0.013112,-0.149434,-0.002536,-0.07,0.13,-0.06,0.07,0.07,0.12,0.15
3,2015-01-31,112.489998,125.580002,112.320000,123.449997,198034100,314.079010,315.838989,171.509995,226.425003,...,0.086899,-0.292848,-0.029629,-0.22,0.05,-0.03,-0.06,-0.26,0.03,0.10
4,2015-02-28,121.839996,123.160004,114.290001,116.160004,125686200,226.490997,256.653015,214.725006,253.828003,...,-0.059052,0.121025,0.056205,-0.28,0.06,-0.14,-0.12,-0.48,-0.22,-0.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,2025-03-31,265.070007,288.390015,265.070007,288.140015,181005700,94248.421875,94429.750000,76624.250000,82548.914062,...,0.094466,-0.021619,-0.055719,-0.02,-0.07,0.09,0.16,0.16,0.04,-0.01
126,2025-04-30,288.540009,317.630005,272.579987,303.769989,354296400,82551.921875,95768.390625,74436.679688,94207.312500,...,0.054244,0.141230,-0.008670,-0.18,-0.02,-0.03,-0.18,-0.41,0.04,-0.08
127,2025-05-31,297.410004,315.619995,291.779999,303.600006,235215100,94212.859375,111970.171875,93399.859375,103998.570312,...,-0.000560,0.103933,0.062845,-0.06,-0.05,-0.10,0.02,-0.18,-0.14,-0.31
128,2025-06-30,309.209991,317.600006,299.890015,304.829987,199565100,105649.812500,110561.421875,99705.750000,107135.335938,...,0.004051,0.030162,0.051386,0.01,-0.08,-0.01,-0.10,-0.18,-0.26,-0.40


In [21]:
file_path = os.path.join("..", "data", "GDP.csv")

#Add GDP data
gdp = pd.read_csv(file_path)
gdp = gdp.rename(columns={"observation_date": "Date"})
gdp["Date"] = pd.to_datetime(gdp["Date"])
gdp = gdp[gdp["Date"] >= "2014-07-01"].copy()
gdp = gdp.set_index("Date")

# Resample quarterly → monthly average
gdp_monthly = gdp.resample("M").ffill() / 3  
gdp_monthly = gdp_monthly.reset_index()

# Add YearMonth column for merging
gdp_monthly["YearMonth"] = gdp_monthly["Date"].dt.to_period("M")


df_analysis = pd.merge(
    df_analysis,
    gdp_monthly[["YearMonth", "GDP"]],   
    on="YearMonth",
    how="left"
)

print(df_analysis.head())
print(df_analysis.info())



        Date   Open_Gold   High_Gold    Low_Gold  Close_Gold  Volume_Gold  \
0 2014-10-31  115.160004  120.500000  111.660004  112.660004    130513900   
1 2014-11-30  112.389999  115.959999  109.669998  112.110001    147594200   
2 2014-12-31  113.800003  118.989998  112.410004  113.580002    153722200   
3 2015-01-31  112.489998  125.580002  112.320000  123.449997    198034100   
4 2015-02-28  121.839996  123.160004  114.290001  116.160004    125686200   

     Open_BTC    High_BTC     Low_BTC   Close_BTC  ...  Close_BTC_Return  \
0  320.389008  411.697998  302.559998  338.321014  ...               NaN   
1  325.569000  457.092987  325.076996  376.446991  ...          0.112692   
2  378.248993  384.037994  304.231995  320.192993  ...         -0.149434   
3  314.079010  315.838989  171.509995  226.425003  ...         -0.292848   
4  226.490997  256.653015  214.725006  253.828003  ...          0.121025   

   Close_SP500_Return   P_I  EU_H   C_H  SO_I CFNAI  CFNAI_MA3  DIFFUSION  \
0  

C:\Users\anais\AppData\Local\Temp\ipykernel_27380\1966912998.py:11: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  gdp_monthly = gdp.resample("M").ffill() / 3


In [22]:
print(df_analysis.isnull().sum())

df_analysis = df_analysis.dropna().reset_index(drop=True)

print(df_analysis.duplicated().sum())

df_analysis.to_csv("analysis.csv", index=False)

Date                  0
Open_Gold             0
High_Gold             0
Low_Gold              0
Close_Gold            0
Volume_Gold           0
Open_BTC              0
High_BTC              0
Low_BTC               0
Close_BTC             0
Volume_BTC            0
Open_SP500            0
High_SP500            0
Low_SP500             0
Close_SP500           0
Volume_SP500          0
YearMonth             0
Close_Gold_Return     1
Close_BTC_Return      1
Close_SP500_Return    1
P_I                   0
EU_H                  0
C_H                   0
SO_I                  0
CFNAI                 0
CFNAI_MA3             0
DIFFUSION             0
GDP                   3
dtype: int64
0


In [23]:

# Reshape to long format
df_long = df_monthly.melt(
    id_vars=['Date'],  
    value_vars=['Close_Gold_Return', 'Close_BTC_Return', 'Close_SP500_Return'],  
    var_name='Asset',    
    value_name='Return'  
)

df_long['Asset'] = df_long['Asset'].str.replace('Close_', '').str.replace('_Return', '')


df_long = df_long.dropna().reset_index(drop=True)


df_long.to_csv('returns_long_format.csv', index=False)

df_long

,Date,Asset,Return
0,2014-11-30,Gold,-0.004882
1,2014-12-31,Gold,0.013112
2,2015-01-31,Gold,0.086899
3,2015-02-28,Gold,-0.059052
4,2015-03-31,Gold,-0.021522
...,...,...,...
391,2025-06-30,SP500,0.051386
392,2025-07-31,SP500,0.023032
393,2025-08-31,SP500,0.020520
394,2025-09-30,SP500,0.035620
